# 로지스틱 회귀
### 데이터 준비

In [4]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish

URLError: <urlopen error [Errno 65] No route to host>

In [ ]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

### KNN 다중분류

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.classes_)

# 5개를 예측
print(kn.predict(test_scaled[:5]))

# 예측한 5개의 확률
# probability: 확률
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


### 로지스틱 회귀(이진 분류)

In [ ]:
# 불리언 인덱싱! 중요한 파이썬 문법이다!
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
# Bream과 Smelt만 뽑는다
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))

print(lr.predict_proba(train_bream_smelt[:5]))

# 0을 도미로, 1을 빙어로 판단한 것임

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']
[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


### 로지스틱 계수 확인

In [ ]:
print(lr.coef_, lr.intercept_)
# a,b,c,d,e,f 값이 나온 것임

dicisions = lr.decision_function(train_bream_smelt)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


### 로지스틱 회귀의 파라미터

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=20, max_iter=1000)
# C: 올라가면 규제가 약해지고, 내려가면 규제가 강해진다 (선형회귀와 반대) || iter: 반복횟수
# max_iter 값은 충분한 학습이 안될거라고 경고를 띄워준다.
# 하지만 C값은 우리가 정해주어야하는 것이므로 경고 띄우진 않음 (하이퍼파라미터)
lr.fit(train_scaled, test_scaled)

proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

ValueError: y should be a 1d array, got an array of shape (40, 5) instead.